In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install unsloth

import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

In [3]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "benjaminzwhite/Llama-3.1-8B_ABSQ-AQSP_LoRA", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

# Inference

- can toggle inference mode

In [4]:
FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [5]:
prompt_template = """Below is a DOCUMENT in which human beings may be expressing themselves about products or services.

Perform a full aspect-based sentiment analysis of the DOCUMENT.
    
Only use sentiment labels that appear in the below list of ALLOWED SENTIMENTS.
    
Only use aspect category labels that appear in the below list of ALLOWED ASPECT CATEGORIES.
    
Use the exact words and spelling found in the DOCUMENT without modification.

Return your answer as a structured JSON object without deviation.

### DOCUMENT:

{manual_add_query}

### ALLOWED SENTIMENTS:

- positive
- negative
- neutral

### ALLOWED ASPECT CATEGORIES:

- food quality
- service general
- restaurant general
- ambience general
- food style_options
- restaurant miscellaneous
- food prices
- restaurant prices
- drinks quality
- drinks style_options
- location general
- drinks prices
- food general

### RESPONSE:
"""

## test single query end to end

- **NOTE:** the base model is 8b, but NOT 8b-instruct **tutorial notebook wasn't clear about this afaict**

- so might be better way of getting response

- here i'm parsing by looking at "what appears after `### RESPONSE:###` O_o

In [7]:
EOS_TOKEN = tokenizer.eos_token

In [8]:
test_query = "I really hated this place's burgers they are the worst i have ever tasted in New York!"

inputs = tokenizer(prompt_template.format(manual_add_query=test_query), return_tensors = "pt").to("cuda")

outputs = model.generate(input_ids = inputs.input_ids,
                         attention_mask = inputs.attention_mask,
                         max_new_tokens = 500)

res = tokenizer.batch_decode(outputs)

# add 0 here - might not work when have several in batch TODO: test with dataset not just 1 sample
start_idx = res[0].index("### RESPONSE:\n")

print(start_idx)

# TODO: fix this better with error handling or if train with chat template understand how it works
generated_absa_text = res[0][start_idx + len("### RESPONSE:\n"):]

# remove the EOS_TOKEN at the end also
generated_absa_text = generated_absa_text.rstrip(EOS_TOKEN)

generated_absa_text

951


'{"aspect_based_sentiment_analysis": [{"opinion term": "burgers", "aspect category": "food quality", "sentiment": "negative", "justification": "worst"}] }'

In [27]:
res

['<|begin_of_text|>Below is a DOCUMENT in which human beings may be expressing themselves about products or services.\n\nPerform a full aspect-based sentiment analysis of the DOCUMENT.\n    \nOnly use sentiment labels that appear in the below list of ALLOWED SENTIMENTS.\n    \nOnly use aspect category labels that appear in the below list of ALLOWED ASPECT CATEGORIES.\n    \nUse the exact words and spelling found in the DOCUMENT without modification.\n\nReturn your answer as a structured JSON object without deviation.\n\n### DOCUMENT:\n\nI really hated this place\'s burgers they are the worst i have ever tasted in New York!\n\n### ALLOWED SENTIMENTS:\n\n- positive\n- negative\n- neutral\n\n### ALLOWED ASPECT CATEGORIES:\n\n- food quality\n- service general\n- restaurant general\n- ambience general\n- food style_options\n- restaurant miscellaneous\n- food prices\n- restaurant prices\n- drinks quality\n- drinks style_options\n- location general\n- drinks prices\n- food general\n\n### RESP

# Evaluation

## Ok loop over test set

- reload the restaurants dataset, do the same processing on it to get same format

In [9]:
from datasets import load_dataset

absa_quad = load_dataset("NEUDM/absa-quad")

README.md:   0%|          | 0.00/2.83k [00:00<?, ?B/s]

Generation/train.jsonl:   0%|          | 0.00/2.00M [00:00<?, ?B/s]

Generation/dev.jsonl:   0%|          | 0.00/503k [00:00<?, ?B/s]

Generation/test.jsonl:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2098 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/525 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1081 [00:00<?, ? examples/s]

In [10]:
import ast
import json

In [33]:
#
# -- iterate over test dataset
#
test_dataset_rows = []

for example in absa_quad["test"].iter(batch_size=1):
    # get the raw text
    example_text = example["input"][0]
    #print(example_text)

    input_text = ast.literal_eval(example_text)[0]
    
    gold_labels = example["output"][0]
    
    #print(gold_labels)
    golds = ast.literal_eval(gold_labels)
    tmp_list_of_quads = []
    for quad in golds:
        #print(quad)
        tmp_d = {}
        tmp_d["opinion term"] = quad[0]
        tmp_d["aspect category"] = quad[1]
        tmp_d["sentiment"] = quad[2]
        tmp_d["justification"] = quad[3]
        
        tmp_list_of_quads.append(tmp_d)
        #print(tmp_d)
    
    test_dataset_rows.append({
        "input_text": input_text,
        "gold_quads": tmp_list_of_quads
    })

In [46]:
import pandas as pd
from datasets import Dataset

# convert to HF dataset
df = pd.DataFrame(test_dataset_rows)

df.sample(3)

,input_text,gold_quads
725,Another plus is the open feel of the restauran...,"[{'opinion term': 'feel', 'aspect category': '..."
562,– Mioposto has a very creative & delicious piz...,"[{'opinion term': 'pizza menu', 'aspect catego..."
501,We asked for beverages and never received them .,"[{'opinion term': 'NULL', 'aspect category': '..."


In [47]:
RANDOM_SEED = 1234

# inference is slow since doing without batches etc - take sample
df_sample = df.sample(n=75, random_state=RANDOM_SEED)

df_sample.head(3)

,input_text,gold_quads
232,But $ 500 for a dinner for two that didn 't in...,"[{'opinion term': 'dinner for two', 'aspect ca..."
62,Great food !,"[{'opinion term': 'food', 'aspect category': '..."
862,– How to describe the best sushi in NYC : hmmm...,"[{'opinion term': 'sushi', 'aspect category': ..."


## iterate over rows and store inference results

In [48]:
#df.shape
df_sample.shape

(75, 2)

In [49]:
print("ITERATING OVER SAMPLE OF 75 TEST EXAMPLES")
print("---")

for index, row in df_sample.iterrows():
    curr_query = row['input_text']
    
    print("Processing input query: ", curr_query)
    
    inputs = tokenizer(prompt_template.format(manual_add_query=curr_query), return_tensors = "pt").to("cuda")

    outputs = model.generate(input_ids = inputs.input_ids,
                             attention_mask = inputs.attention_mask,
                             max_new_tokens = 512)

    res = tokenizer.batch_decode(outputs)
    
    raw_model_completion = res[0]
    
    #df.loc[index, "raw_model_completion"] = raw_model_completion
    # update df_sample ---- redo with full dataset later TODO
    df_sample.loc[index, "raw_model_completion"] = raw_model_completion
    

ITERATING OVER SAMPLE OF 75 TEST EXAMPLES
---
Processing input query:  But $ 500 for a dinner for two that didn 't include Wine ?
Processing input query:  Great food !
Processing input query:  – How to describe the best sushi in NYC : hmmmm , delicious , amazing , fantastic , suculent , perfect , nah , all of the above .
Processing input query:  Servers are all different , Greg is my favorite .
Processing input query:  The pancakes were certainly inventive but $ 8.50 for 3 - 6 `` pancakes ( one of them was more like 5 `` ) in the pancake flight ( sample of 3 different pancakes ) is well over-priced .
Processing input query:  Brunch at Murphy 's is to die for , my specialty ... egg white omelet , the food is always freshly prepared .
Processing input query:  Food was good and cheap .
Processing input query:  Would NEVER go back
Processing input query:  To the owners of Open Sesame ... Bravo ... I ca n't wait to come back to dine at your restaurant !
Processing input query:  Overall the 

In [51]:
df_sample.tail(3)

,input_text,gold_quads,raw_model_completion
124,I came across Village Underground by accident ...,"[{'opinion term': 'Village Underground', 'aspe...",<|begin_of_text|>Below is a DOCUMENT in which ...
899,"While this diner had reasonably good food , th...","[{'opinion term': 'food', 'aspect category': '...",<|begin_of_text|>Below is a DOCUMENT in which ...
839,A brief conversation with the manager at the e...,"[{'opinion term': 'manager', 'aspect category'...",<|begin_of_text|>Below is a DOCUMENT in which ...


# Export to JSON for later

In [52]:
with open('test_inference_75_samples.json', 'w') as file:
    json.dump(df_sample.to_json(), file)

# Explore

In [53]:
df_sample

,input_text,gold_quads,raw_model_completion
232,But $ 500 for a dinner for two that didn 't in...,"[{'opinion term': 'dinner for two', 'aspect ca...",<|begin_of_text|>Below is a DOCUMENT in which ...
62,Great food !,"[{'opinion term': 'food', 'aspect category': '...",<|begin_of_text|>Below is a DOCUMENT in which ...
862,– How to describe the best sushi in NYC : hmmm...,"[{'opinion term': 'sushi', 'aspect category': ...",<|begin_of_text|>Below is a DOCUMENT in which ...
76,"Servers are all different , Greg is my favorite .","[{'opinion term': 'Greg', 'aspect category': '...",<|begin_of_text|>Below is a DOCUMENT in which ...
966,The pancakes were certainly inventive but $ 8....,"[{'opinion term': 'pancakes', 'aspect category...",<|begin_of_text|>Below is a DOCUMENT in which ...
...,...,...,...
732,The best Chuwam Mushi I have ever had .,"[{'opinion term': 'Chuwam Mushi', 'aspect cate...",<|begin_of_text|>Below is a DOCUMENT in which ...
919,Amazing Spanish Mackeral special appetizer and...,[{'opinion term': 'Spanish Mackeral special ap...,<|begin_of_text|>Below is a DOCUMENT in which ...
124,I came across Village Underground by accident ...,"[{'opinion term': 'Village Underground', 'aspe...",<|begin_of_text|>Below is a DOCUMENT in which ...
899,"While this diner had reasonably good food , th...","[{'opinion term': 'food', 'aspect category': '...",<|begin_of_text|>Below is a DOCUMENT in which ...
